In [3]:
import os
import argparse
import cv2
import math
from glob import glob
# Standard packages
import tempfile
import warnings
import urllib
import shutil
#import requests
from PIL import Image
from io import BytesIO
import tqdm
from tqdm.notebook import tqdm_notebook
from skimage.metrics import structural_similarity as compare_ssim
#import imutils
import random
import numpy as np
import psutil
import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
import rioxarray
import re
import rtree
import pickle
import pyproj
import shapely
from shapely.geometry import Polygon, Point
from shapely.ops import transform
#from cartopy import crs
import collections
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
#Parsing/Modifying XML
from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom
import data_eng.az_proc as ap
import data_eng.form_calcs as fc


Please wait a moment while I gather a list of all available modules...



C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\pkgutil.py:92: UserWarning: The numpy.array_api submodule is still experimental. See NEP 47.
  __import__(info.name)
C:\Users\cscro\.virtualenvs\AST_dataset\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


IPython             cligj               multiprocessing     soupsieve
PIL                 cmath               munch               sqlite3
__future__          cmd                 nbclassic           sre_compile
_abc                code                nbclient            sre_constants
_aix_support        codecs              nbconvert           sre_parse
_argon2_cffi_bindings codeop              nbformat            ssl
_ast                collections         nest_asyncio        sspi
_asyncio            colorama            netbios             sspicon
_bisect             colorsys            netrc               stack_data
_blake2             commctrl            networkx            stat
_bootsubprocess     compare_imgs_by_tile nntplib             statistics
_bz2                compileall          notebook            string
_cffi_backend       concurrent          notebook_shim       stringprep
_codecs             configparser        nt                  struct
_codecs_cn          conftest      

C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\pkgutil.py:92: UserWarning: viewer requires Qt
  __import__(info.name)
C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.2544.0_x64__qbz5n2kfra8p0\lib\pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)


In [13]:
parent_dir = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//sample_compile_dir"
xml_folder_name = "chips_positive_corrected_xml"
tile_dir = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//tiles"
tile_level_annotation_dir = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//tile_level_annotations"
tile_level_annotation_dataset_filename = "tile_level_annotation"
item_dim = 512
distance_limit=int(5)
states_gpds_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/US Political Boundaries/US State Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"

In [11]:
tile_characteristics, image_characteristics = fc.image_tile_characteristics(parent_dir, tile_dir, xml_folder_name)

100%|██████████| 3/3 [00:14<00:00,  4.70s/it]


In [14]:
tiles_xml_dir = os.path.join(tile_level_annotation_dir, "tiles_xml")
os.makedirs(tiles_xml_dir, exist_ok=True)
fc.generate_tile_xmls(parent_dir, tile_dir, xml_folder_name, tiles_xml_dir, item_dim)

100%|██████████| 3/3 [00:43<00:00, 14.35s/it]


In [18]:
np.linspace(1,512,512) #row - 0
np.linspace(512+1,512*2,512) #row - 1
np.linspace(512*2+1,512*3,512) #row - 3

#a + value
#512 + value
# row(512) + value
# row(512

array([1025., 1026., 1027., 1028., 1029., 1030., 1031., 1032., 1033.,
       1034., 1035., 1036., 1037., 1038., 1039., 1040., 1041., 1042.,
       1043., 1044., 1045., 1046., 1047., 1048., 1049., 1050., 1051.,
       1052., 1053., 1054., 1055., 1056., 1057., 1058., 1059., 1060.,
       1061., 1062., 1063., 1064., 1065., 1066., 1067., 1068., 1069.,
       1070., 1071., 1072., 1073., 1074., 1075., 1076., 1077., 1078.,
       1079., 1080., 1081., 1082., 1083., 1084., 1085., 1086., 1087.,
       1088., 1089., 1090., 1091., 1092., 1093., 1094., 1095., 1096.,
       1097., 1098., 1099., 1100., 1101., 1102., 1103., 1104., 1105.,
       1106., 1107., 1108., 1109., 1110., 1111., 1112., 1113., 1114.,
       1115., 1116., 1117., 1118., 1119., 1120., 1121., 1122., 1123.,
       1124., 1125., 1126., 1127., 1128., 1129., 1130., 1131., 1132.,
       1133., 1134., 1135., 1136., 1137., 1138., 1139., 1140., 1141.,
       1142., 1143., 1144., 1145., 1146., 1147., 1148., 1149., 1150.,
       1151., 1152.,

In [4]:
#by_tile_folder = os.listdir(args.complete_dir)
complete_dir = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//temp/rechip"

In [2]:
import numpy as np
np.linspace(1,628,628)

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128., 129., 130., 131., 132.,
       133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
       144., 145., 146., 147., 148., 149., 150., 15

In [9]:
xml_paths = glob(complete_dir + "/**/*.xml", recursive = True)

for xml_path in xml_paths:
    fc.reclassify_narrow_closed_roof_and_closed_roof_tanks(xml_path)

In [16]:
x =random.sample(range(100), 40)  
y =random.sample(x, 20)  
len(y)


20

In [20]:
np.random.seed(0)
train_FileNames, val_FileNames, test_FileNames = np.split(np.array(list(range(100))),
                                                          [int(len(list(range(100)))*0.7), int(len(list(range(100)))*0.)])

In [44]:
img_ids = os.listdir(os.path.join("//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//complete_dataset_/chips_positive"))
num_imgs = len(img_ids)  
num_train_imgs = math.ceil(num_imgs * 0.75)  
num_val_imgs = math.ceil(num_train_imgs * 0.1)  
num_test_imgs = num_imgs - num_train_imgs
#randomly sample images to be in the train/val and test sets
random.seed(1)
train_img_ids = random.sample(img_ids, num_train_imgs)  
val_img_ids = random.sample(train_img_ids, num_val_imgs)  
test_img_ids = list(set(img_ids) - set(train_img_ids) - set(val_img_ids))

In [41]:
len(np.unique(train_img_ids))

17154

In [42]:
len(np.unique(val_img_ids))

1716

In [45]:
len(np.unique(test_img_ids))

5718

## File Paths

In [4]:

#create folder to hold tiles in completed dataset
#load tiles and tile xmls
tiles_dir = os.path.join(parent_directory,"complete_dataset","tiles")
os.makedirs(tiles_dir, exist_ok=True)

tiles_xml_dir = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_xml_dir, exist_ok=True)
tiles_xml_list = os.listdir(tiles_xml_dir)

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)
rechipped_image_path = os.path.join(parent_directory,"temp/rechip")

folders_of_images_xmls_by_tile = os.listdir(rechipped_image_path)
images_and_xmls_by_tile_path = rechipped_image_path

tile_name_tile_url_complete_array_path = "image_download_azure/tile_name_tile_url_complete_array.csv"
tile_name_tile_url_complete_array = pd.read_csv(tile_name_tile_url_complete_array_path)
tile_name_tile_url_complete_array = tile_name_tile_url_complete_array.to_numpy()
tile_name_tile_url_complete_array = tile_name_tile_url_complete_array[:,(1,2)] #remove index

FileNotFoundError: [WinError 3] The system cannot find the path specified: '//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//temp/rechip'

# Add chips to rechip folder
Add all chips corresponding to a tile to the 'chips' folder for each tile

In [3]:
#for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
#    add_chips_to_chip_folders(rechipped_image_path, tile_name)

# Correct object labels

## Fix inconsistent labels 

In [ ]:
for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
    #specify image/xml paths for each tile
    positive_xml_dir = os.path.join(images_and_xmls_by_tile_path, tile_name, "chips_positive_xml")
    fc.correct_inconsistent_labels_xml(positive_xml_dir)

## Reclassify narrow closed roof tanks and closed roof tanks by size

In [ ]:
for tile in tqdm.tqdm(folders_of_images_xmls_by_tile): #iterate over tile folders
    tile_annotation_path = os.path.join(images_and_xmls_by_tile_path, tile, "chips_positive_xml")
    for xml in os.listdir(tile_annotation_path):
        xml_path = os.path.join(tile_annotation_path, xml)
        fc.reclassify_narrow_closed_roof_and_closed_roof_tanks(xml_path)

# Generate table of characteristics for tiles/images

In [5]:
#tile_characteristics, image_characteristics = fc.image_tile_characteristics(rechipped_image_path, tiles_dir)#, tile_name_tile_url_complete_array)

100%|██████████████████████████████████████████████████████████████████████████████| 2139/2139 [25:57<00:00,  1.37it/s]


In [ ]:
#tile_dimensions_and_utm_coords()

In [6]:
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_characteristics = pd.read_csv("tile_characteristics.csv")

# Create tile level annotations

## Generate tile level XMLs

## Merge neighboring bounding boxes within each tile
References:
https://answers.opencv.org/question/231263/merging-nearby-rectanglesedited/
https://stackoverflow.com/questions/55593506/merge-the-bounding-boxes-near-by-into-one

In [7]:
distance_limit = 5
tile_database = fc.merge_tile_annotations(tile_characteristics, tiles_xml_dir, distance_limit = 5)
#merged_tile_database =  tile_database[tile_database["merged_bbox"] == True]

100%|██████████████████████████████████████████████████████████████████████████████| 2120/2120 [54:39<00:00,  1.55s/it]


# Clip storm surge data and append to tile level annotation database

#sc_slosh_inundation_map_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/Natural Hazards Data/NHC Potential Storm Surge Flood Map/SC_SLOSH_MOM_Inundation"
#tile_database = fc.identify_inundation_for_tanks(tile_database, sc_slosh_inundation_map_path)

# Add in state

In [8]:
states_gpds_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/US Political Boundaries/US State Boundaries/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
tile_database = fc.identify_state_name_for_each_state(states_gpds_path, tile_database)
#check issues in state list
#state_list = state_list[state_list!=None]
#np.unique(state_list)

98169it [03:08, 519.82it/s] 


In [16]:
#https://localcoder.org/merge-the-bounding-boxes-near-by-into-one

In [12]:
x = "Y:/rechip/m_2508015_se_17_060_20191120/chips_positive_xml/m_2508015_se_17_060_20191120_23_12.xml"


'20191120'

# Save tile dabasebase

In [9]:
output_filepath = os.path.join(parent_directory,"complete_dataset","tile_level_annotations")
output_filename = 'tile_level_annotations'
fc.write_gdf(tile_database, output_filepath, output_filename)

C:\Users\rapiduser\cred\AST_dataset\data_download_and_preprocessing\data_eng\form_calcs.py:1095: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_shapefile.to_file(os.path.join(output_filepath,output_filename+".shp"))


In [4]:
#fc.generate_tile_xmls(rechipped_image_path, tiles_dir, tiles_xml_dir, int(512))